# Big Data, Programa de Ciencia de los Datos
## Tarea #1

* Esteban Sáenz Villalobos (**esaenz7@gmail.com**)
* Entrega: 08 de agosto 2021, 22:00.
* Observaciones: Trabajo elaborado desde Google Colab. Ejecutar cada celda de código de forma secuencial.

---

In [1]:
from IPython.display import Javascript

In [2]:
'''
Instalación de PySpark en Colab
'''
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import requests, os
from bs4 import BeautifulSoup
#obtener las versiones de spark e instalar la última disponile
soup = BeautifulSoup(requests.get('https://downloads.apache.org/spark/').text)
link_files = []
[link_files.append(link.get('href')) for link in soup.find_all('a')]
spark_link = [x for x in link_files if 'spark' in x]
ver_spark = spark_link[-1][:-1]
os.system(f"wget -q https://www-us.apache.org/dist/spark/{ver_spark}/{ver_spark}-bin-hadoop3.2.tgz")
os.system(f"tar xf {ver_spark}-bin-hadoop2.7.tgz")
#instalar pyspark
!pip install -q pyspark
!pip --version
!pyspark --version
#instalar pytests
!pip install -q pytest pytest-sugar
!pytest --version
# from pathlib import Path
# if Path.cwd().name != 'tdd':
#     %mkdir tdd
#     %cd tdd
# %pwd
# %rm *.py
#---

     |████████████████████████████████| 212.4 MB 69 kB/s 
     |████████████████████████████████| 198 kB 51.6 MB/s 
pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/
                        
Using Scala version 2.12.10, OpenJDK 64-Bit Server VM, 11.0.11
Branch HEAD
Compiled by user centos on 2021-05-24T04:27:48Z
Revision de351e30a90dd988b133b3d00fa6218bfcaba8b8
Url https://github.com/apache/spark
Type --help for more information.
This is pytest version 3.6.4, imported from /usr/local/lib/python2.7/dist-packages/pytest.pyc


In [3]:
'''
https://drive.google.com/file/d/1-GyCL-ylOmAx8BWzKLQpqhx_LOwD3ane/view?usp=sharing, 
https://drive.google.com/file/d/12EvDT_cWj7beSEEjBOyNWABgBoNYYEzb/view?usp=sharing, 
https://drive.google.com/file/d/186Sy-IVW5OcSTPXL0kRWYTulkSGVj8Tu/view?usp=sharing, 
https://drive.google.com/file/d/1gK6CYRvhY4nx-zIj_U1vz5xCwo3dsYnS/view?usp=sharing, 
https://drive.google.com/file/d/1qq3rUNOrwOBuWiiC8sY0IWiZinqupwNQ/view?usp=sharing
'''
!pip install -q gdown
!gdown https://drive.google.com/uc?id=1-GyCL-ylOmAx8BWzKLQpqhx_LOwD3ane
!gdown https://drive.google.com/uc?id=12EvDT_cWj7beSEEjBOyNWABgBoNYYEzb
!gdown https://drive.google.com/uc?id=186Sy-IVW5OcSTPXL0kRWYTulkSGVj8Tu
!gdown https://drive.google.com/uc?id=1gK6CYRvhY4nx-zIj_U1vz5xCwo3dsYnS
!gdown https://drive.google.com/uc?id=1qq3rUNOrwOBuWiiC8sY0IWiZinqupwNQ

Downloading...
From: https://drive.google.com/uc?id=1-GyCL-ylOmAx8BWzKLQpqhx_LOwD3ane
To: /content/persona03.json
100% 1.57k/1.57k [00:00<00:00, 2.32MB/s]
Downloading...
From: https://drive.google.com/uc?id=12EvDT_cWj7beSEEjBOyNWABgBoNYYEzb
To: /content/persona02.json
100% 1.57k/1.57k [00:00<00:00, 2.75MB/s]
Downloading...
From: https://drive.google.com/uc?id=186Sy-IVW5OcSTPXL0kRWYTulkSGVj8Tu
To: /content/persona05.json
100% 1.57k/1.57k [00:00<00:00, 2.69MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gK6CYRvhY4nx-zIj_U1vz5xCwo3dsYnS
To: /content/persona01.json
100% 1.57k/1.57k [00:00<00:00, 3.76MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qq3rUNOrwOBuWiiC8sY0IWiZinqupwNQ
To: /content/persona04.json
100% 1.57k/1.57k [00:00<00:00, 2.47MB/s]


In [4]:
%%file procesamientodatos.py
'''
Nombre de archivo:
  +procesamientodatos.py
Descripción: 
  +Librería con funciones para el procesamiento de los datos
Métodos:
  |--+cargar_datos
  |--+generar_tablas
  |--+almacenar_tablas
'''

#librerías necesarias
import sys, os, datetime
from pyspark.sql import SparkSession, functions as F, window as W
from pyspark.sql.types import (DateType, IntegerType, FloatType, DoubleType, LongType, StringType, StructField, StructType, TimestampType)

#sesión de spark
spark = SparkSession.builder\
        .master("local")\
        .appName("App#1")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

#función para carga de datos (lista de archivos .json)
def cargar_datos(files=[]):
  try:
    #lectura de archivos .json
    df1 = spark.read.json(files, multiLine=True)
    #se realizan las transformaciones necesarias para obtener cada uno de los elementos del esquema
    df1 = df1.withColumn('viajes', F.explode(F.col('viajes'))).select('identificador','viajes.*').orderBy('identificador')
    df1.collect()
    return [df1]
  except Exception as e:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print(exc_type, os.path.split(exc_tb.tb_frame.f_code.co_filename)[1], exc_tb.tb_lineno, exc_obj)

#función para generar las tablas con los resultados de los datos procesados
def generar_tablas(df=[]):
  try:
    #se crean dataframes temporales que sirven como tablas intermedias para el filtrado y agregación de los datos
    df1a = df[0].withColumnRenamed('codigo_postal_origen','codigo_postal').withColumn('tipo', F.lit('origen'))\
    .groupBy('codigo_postal', 'tipo').agg(F.count('codigo_postal').alias('cantidad_viajes'), F.sum(F.col('kilometros')*F.col('precio_kilometro')).alias('ingresos'))
    df1b = df[0].withColumnRenamed('codigo_postal_destino','codigo_postal').withColumn('tipo', F.lit('destino'))\
    .groupBy('codigo_postal', 'tipo').agg(F.count('codigo_postal').alias('cantidad_viajes'), F.sum(F.col('kilometros')*F.col('precio_kilometro')).alias('ingresos'))
    df1c = df[0].select('identificador', 'kilometros', 'precio_kilometro')\
    .groupBy('identificador').agg(F.sum('kilometros').alias('cantidad_kms'), F.sum(F.col('kilometros')*F.col('precio_kilometro')).alias('ingresos'))
    #tabla correspondiente a la cantidad de viajes por código postal
    df2 = df1a.union(df1b).select('codigo_postal', 'tipo', 'cantidad_viajes').orderBy(F.col('codigo_postal'), F.col('tipo').desc())
    #tabla correspondiente a los ingresos totales por código postal
    df3 = df1a.union(df1b).select('codigo_postal', 'tipo', F.round('ingresos',2).alias('ingresos')).orderBy(F.col('codigo_postal'), F.col('tipo').desc())
    #tabla correspondiente a la cantidad de kms e ingresos por identificador de conductor
    df4 = df1c.select('identificador', F.round('cantidad_kms',2).alias('cantidad_kms'), F.round('ingresos',2).alias('ingresos')).orderBy(F.col('identificador'))
    #tabla correspondiente a métricas particulares
    data = [('persona_con_mas_kilometros', df4.groupBy('identificador').agg(F.max('cantidad_kms')).orderBy(F.col('max(cantidad_kms)').desc()).collect()[0][0]),\
            ('persona_con_mas_ingresos', df4.groupBy('identificador').agg(F.max('ingresos')).orderBy(F.col('max(ingresos)').desc()).collect()[0][0]),\
            ('percentil_25', df4.select(F.percentile_approx('ingresos', .25)).collect()[0][0]),\
            ('percentil_50', df4.select(F.percentile_approx('ingresos', .50)).collect()[0][0]),\
            ('percentil_75', df4.select(F.percentile_approx('ingresos', .75)).collect()[0][0]),\
            ('codigo_postal_origen_con_mas_ingresos', df1a.groupBy('codigo_postal').agg(F.max('ingresos')).orderBy(F.col('max(ingresos)').desc()).collect()[0][0]),\
            ('codigo_postal_destino_con_mas_ingresos', df1b.groupBy('codigo_postal').agg(F.max('ingresos')).orderBy(F.col('max(ingresos)').desc()).collect()[0][0])]
    schema = StructType(\
                        [StructField('tipo_metrica',StringType()),
                         StructField('valor',StringType()),])
    df5 = spark.createDataFrame(data, schema)
    #se agregan los dataframes a una lista para la iteración
    proceso = [df2, df3, df5]
    #por medio de las funciones list-map-lambda se ejecutan las operaciones iterando sobre los dataframes creados
    list(map(lambda x: {x.printSchema(), x.show(50)}, proceso)) #se despliegan el esquema y los datos correspondientes a cada tabla
    return proceso
  except Exception as e:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print(exc_type, os.path.split(exc_tb.tb_frame.f_code.co_filename)[1], exc_tb.tb_lineno, exc_obj)

#función para almacenar los dataframes en formato .csv
def almacenar_tablas(df=[], files_name=[]):
  try:
    #escritura de los archivos
    csv_files=[]
    if (len(df)==len(files_name)):
      #se ejecutan las operaciones de escritura iterando sobre cada objeto
      list(map(lambda x, y: {x.write.csv(y, mode='overwrite')}, df, files_name))
      #se ejecuta una función de comprobación, leyendo cada archivo creado
      [csv_files.append(spark.read.csv(files_name[i])) for i in range(len(files_name))]
      if csv_files: print('Tablas almacenadas: '+ str(files_name))
    return csv_files
  except Exception as e:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print(exc_type, os.path.split(exc_tb.tb_frame.f_code.co_filename)[1], exc_tb.tb_lineno, exc_obj)
#

Writing procesamientodatos.py


In [5]:
%%file programaestudiante.py
'''
Nombre de archivo:
  +programaestudiante.py
Descripción: 
  +Archivo principal (main) para la ejecución del programa
'''

#librerías necesarias
import sys, os
from procesamientodatos import *

#llamado a la función ejecutar_proceso
dfinicial = cargar_datos(files=sys.argv)
proceso = generar_tablas(dfinicial)
archivos = almacenar_tablas(proceso, ['total_viajes.csv', 'total_ingresos.csv', 'metricas.csv'])
#

Writing programaestudiante.py


In [6]:
%%file conftest.py
'''
Nombre de archivo:
  +conftest.py
Descripción: 
  +Archivo para definición del contexto para la ejecución de las pruebas
'''

#librerías necesarias
import pytest
from procesamientodatos import *

#definición de los parámetros (fixtures) de pruebas según las diferentes etapas de ejecución del programa
#
#***resultados actuales*** obtenidos durante la ejecución del programa de forma regular
#
#parámetro para prueba de carga de datos
dfinicial = cargar_datos('persona*.json')
@pytest.fixture
def tstage1():
  return dfinicial
#parámetro para prueba de procesamiento de datos
proceso = generar_tablas(dfinicial)
@pytest.fixture
def tstage2():
  return proceso
#parámetro para prueba de almacenamiento
archivos = almacenar_tablas(proceso, ['total_viajes.csv', 'total_ingresos.csv', 'metricas.csv'])
@pytest.fixture
def tstage3():
  return archivos
#
#***resultados actuales*** obtenidos durante la ejecución del programa alterando valores en los datos
#
#parámetro para prueba completa con valores en cero (kilometros)
@pytest.fixture
def tzerodata():
  dfzerodata = dfinicial[0]
  #se reemplazan algunos valores de la columna "kilometros" por 0
  dfzerodata = dfzerodata.withColumn('precio_kilometro', F.when(F.col('kilometros')<5, 0).otherwise(F.col('precio_kilometro')))
  #se procesan los datos con la función "generar_tablas()"
  rzerodata = generar_tablas([dfzerodata])
  return rzerodata[1]
#parámetro para prueba completa con valores nulos (codigo_postal_*)
@pytest.fixture
def tmissdata():
  dfmissdata = dfinicial[0]
  #se reemplazan algunos valores de las columnas "codigo_postal_destino" y "codigo_postal_destino" por nulos
  dfmissdata = dfmissdata.withColumn('codigo_postal_destino', F.when(F.col('kilometros')<5, '').otherwise(F.col('codigo_postal_destino')))
  dfmissdata = dfmissdata.withColumn('codigo_postal_origen', F.when(F.col('kilometros')>20, '').otherwise(F.col('codigo_postal_origen')))
  #se procesan los datos con la función "generar_tablas()"
  rmissdata = generar_tablas([dfmissdata])
  return rmissdata[0]
#parámetro para prueba completa con filas eliminadas (filtradas)
@pytest.fixture
def tdeldata():
  dfdeldata = dfinicial[0]
  #se eliminan algunas filas por completo (filtro por "precio_kilometro" > 450)
  dfdeldata = dfdeldata.filter('precio_kilometro > 450')
  #se procesan los datos con la función "generar_tablas()"
  rdeldata = generar_tablas([dfdeldata])
  return rdeldata[0]

#***resultados esperados*** para pruebas de comparación
#
#parámetro para prueba de resultados finales completos
@pytest.fixture
def tstage2_expected():
  data1 = [('10101', 'origen', 6),
          ('10101', 'destino', 2),
          ('10201', 'origen', 4),
          ('10201', 'destino', 2),
          ('10301', 'origen', 2),
          ('10301', 'destino', 4),
          ('10601', 'destino', 2),
          ('10701', 'destino', 3),
          ('10801', 'destino', 1),
          ('11001', 'origen', 3),
          ('11101', 'destino', 6),
          ('11301', 'origen', 1),
          ('11301', 'destino', 3),
          ('11401', 'origen', 3),
          ('11501', 'origen', 3),
          ('11801', 'origen', 3),
          ('11801', 'destino', 4),
          ('20101', 'origen', 4),
          ('30101', 'origen', 5),
          ('30101', 'destino', 3),
          ('40101', 'destino', 2),
          ('40201', 'origen', 4),
          ('40301', 'origen', 5),
          ('40401', 'origen', 3),
          ('40401', 'destino', 5),
          ('40501', 'destino', 8),
          ('40701', 'origen', 3),
          ('40701', 'destino', 3),
          ('40801', 'origen', 1),
          ('40801', 'destino', 2)]
  data2 = [('10101', 'origen', 32703.0),
          ('10101', 'destino', 3276.0),
          ('10201', 'origen', 7055.0),
          ('10201', 'destino', 3645.0),
          ('10301', 'origen', 3492.5),
          ('10301', 'destino', 18571.0),
          ('10601', 'destino', 1920.0),
          ('10701', 'destino', 2545.5),
          ('10801', 'destino', 14670.0),
          ('11001', 'origen', 6115.0),
          ('11101', 'destino', 14734.0),
          ('11301', 'origen', 5322.0),
          ('11301', 'destino', 19721.0),
          ('11401', 'origen', 7395.0),
          ('11501', 'origen', 12216.0),
          ('11801', 'origen', 9708.0),
          ('11801', 'destino', 16446.5),
          ('20101', 'origen', 17651.5),
          ('30101', 'origen', 18368.5),
          ('30101', 'destino', 11412.0),
          ('40101', 'destino', 11944.0),
          ('40201', 'origen', 19432.0),
          ('40301', 'origen', 18010.5),
          ('40401', 'origen', 22559.5),
          ('40401', 'destino', 20627.0),
          ('40501', 'destino', 46159.0),
          ('40701', 'origen', 28654.0),
          ('40701', 'destino', 15537.0),
          ('40801', 'origen', 372.0),
          ('40801', 'destino', 7846.5)]
  data3 = [('persona_con_mas_kilometros', '01004'),
          ('persona_con_mas_ingresos', '01004'),
          ('percentil_25', '37148.5'),
          ('percentil_50', '38619.0'),
          ('percentil_75', '45756.0'),
          ('codigo_postal_origen_con_mas_ingresos', '10101'),
          ('codigo_postal_destino_con_mas_ingresos', '40501')]
  stage2_expected = [spark.createDataFrame(data1), spark.createDataFrame(data2), spark.createDataFrame(data3)]
  return stage2_expected
#parámetro para prueba completa con valores en cero
@pytest.fixture
def tzerodata_expected():
  data = [('10101', 'origen', 30489.0),
          ('10101', 'destino', 0.0),
          ('10201', 'origen', 5374.0),
          ('10201', 'destino', 2934.0),
          ('10301', 'origen', 2628.0),
          ('10301', 'destino', 17630.5),
          ('10601', 'destino', 0.0),
          ('10701', 'destino', 0.0),
          ('10801', 'destino', 14670.0),
          ('11001', 'origen', 4655.0),
          ('11101', 'destino', 13231.5),
          ('11301', 'origen', 5322.0),
          ('11301', 'destino', 17130.5),
          ('11401', 'origen', 6075.0),
          ('11501', 'origen', 8563.5),
          ('11801', 'origen', 8997.0),
          ('11801', 'destino', 13371.5),
          ('20101', 'origen', 16711.0),
          ('30101', 'origen', 16173.0),
          ('30101', 'destino', 11412.0),
          ('40101', 'destino', 11944.0),
          ('40201', 'origen', 19432.0),
          ('40301', 'origen', 15796.5),
          ('40401', 'origen', 22559.5),
          ('40401', 'destino', 19562.0),
          ('40501', 'destino', 46159.0),
          ('40701', 'origen', 27793.0),
          ('40701', 'destino', 14677.0),
          ('40801', 'origen', 0.0),
          ('40801', 'destino', 7846.5)]
  schema = None
  result = spark.createDataFrame(data, schema)
  return result
#parámetro para prueba completa con valores nulos
@pytest.fixture
def tmissdata_expected():
  data = [('', 'origen', 5),
          ('', 'destino', 16),
          ('10101', 'origen', 5),
          ('10201', 'origen', 4),
          ('10201', 'destino', 1),
          ('10301', 'origen', 2),
          ('10301', 'destino', 3),
          ('10801', 'destino', 1),
          ('11001', 'origen', 3),
          ('11101', 'destino', 4),
          ('11301', 'origen', 1),
          ('11301', 'destino', 2),
          ('11401', 'origen', 3),
          ('11501', 'origen', 3),
          ('11801', 'origen', 3),
          ('11801', 'destino', 2),
          ('20101', 'origen', 4),
          ('30101', 'origen', 4),
          ('30101', 'destino', 3),
          ('40101', 'destino', 2),
          ('40201', 'origen', 4),
          ('40301', 'origen', 5),
          ('40401', 'origen', 2),
          ('40401', 'destino', 4),
          ('40501', 'destino', 8),
          ('40701', 'origen', 1),
          ('40701', 'destino', 2),
          ('40801', 'origen', 1),
          ('40801', 'destino', 2)]
  schema = None
  result = spark.createDataFrame(data, schema)
  return result
#parámetro para prueba completa con filas eliminadas
@pytest.fixture
def tdeldata_expected():
  data = [('10101', 'origen', 2),
          ('10101', 'destino', 1),
          ('10201', 'origen', 1),
          ('10301', 'destino', 2),
          ('10701', 'destino', 1),
          ('10801', 'destino', 1),
          ('11001', 'origen', 2),
          ('11101', 'destino', 1),
          ('11301', 'destino', 1),
          ('11401', 'origen', 1),
          ('11501', 'origen', 2),
          ('11801', 'origen', 1),
          ('11801', 'destino', 2),
          ('20101', 'origen', 2),
          ('30101', 'destino', 2),
          ('40201', 'origen', 1),
          ('40301', 'origen', 1),
          ('40401', 'origen', 1),
          ('40501', 'destino', 3),
          ('40701', 'origen', 2),
          ('40701', 'destino', 2)]
  schema = None
  result = spark.createDataFrame(data, schema)
  return result
#

Writing conftest.py


In [7]:
%%file test_programaestudiante.py
'''
Nombre de archivo:
  +test_programaestudiante.py
Descripción: 
  +Archivo para la ejecuión de las pruebas del programa
'''

#librerías necesarias
import pytest
from pyspark.sql import functions as F

#pruebas según las diferentes etapas de ejecución del programa (stages)
#se comparan los valores actuales obtenidos desde los "fixtures" con los valores esperados según cada condición
#
#prueba de carga de datos
def test_stage1(tstage1):
  assert type(tstage1) == list
  assert len(tstage1) == 1
  assert tstage1[0].count() == 50
  assert str(tstage1[0].dtypes) == "[('identificador', 'string'), ('codigo_postal_destino', 'string'), ('codigo_postal_origen', 'string'), ('kilometros', 'string'), ('precio_kilometro', 'string')]"
#prueba de procesamiento de datos
def test_stage2(tstage2, tstage2_expected):
  assert type(tstage2) == list
  assert len(tstage2) == 3
  assert tstage2[0].count() == 30
  assert tstage2[1].count() == 30
  assert tstage2[2].count() == 7
  assert str(tstage2[0].dtypes) == "[('codigo_postal', 'string'), ('tipo', 'string'), ('cantidad_viajes', 'bigint')]"
  assert str(tstage2[1].dtypes) == "[('codigo_postal', 'string'), ('tipo', 'string'), ('ingresos', 'double')]"
  assert str(tstage2[2].dtypes) == "[('tipo_metrica', 'string'), ('valor', 'string')]"
  #el método de comparación entre el dataframe esperado y el actual retorna la cantidad de filas distintas entre ambos, por su forma se implementa en ambas vías, actual vs esperado y esperado vs actual
  assert tstage2_expected[0].exceptAll(tstage2[0]).count() == 0
  assert tstage2[0].exceptAll(tstage2_expected[0]).count() == 0
  assert tstage2_expected[1].exceptAll(tstage2[1]).count() == 0
  assert tstage2[1].exceptAll(tstage2_expected[1]).count() == 0
  #es un método simple y funciona sin importar el orden de los datos, ejem. ordenando los datos de forma aleatoria:
  tstage2_shuffle = tstage2[2].orderBy(F.rand()) #df actual
  tstage2_expected_shuffle = tstage2_expected[2].orderBy(F.rand()) #df esperado
  assert tstage2_expected_shuffle.exceptAll(tstage2_shuffle).count() == 0
  assert tstage2_shuffle.exceptAll(tstage2_expected_shuffle).count() == 0  

#prueba de almacenamiento
def test_stage3(tstage3):
  assert type(tstage3) == list
  assert len(tstage3) == 3
  assert tstage3[0].count() == 30
  assert tstage3[1].count() == 30
  assert tstage3[2].count() == 7
#prueba completa con valores en cero
def test_zerodata(tzerodata, tzerodata_expected):
  assert tzerodata.count() == 30
  assert tzerodata_expected.exceptAll(tzerodata).count() == 0
  assert tzerodata.exceptAll(tzerodata_expected).count() == 0
#prueba completa con valores nulos
def test_missdata(tmissdata, tmissdata_expected):
  assert tmissdata.count() == 29
  assert tmissdata_expected.exceptAll(tmissdata).count() == 0
  assert tmissdata.exceptAll(tmissdata_expected).count() == 0
#prueba completa con filas eliminadas
def test_deldata(tdeldata, tdeldata_expected):
  tdeldata.show()
  tdeldata_expected.show()
  assert tdeldata.count() == 21
  assert tdeldata_expected.exceptAll(tdeldata).count() == 0
  assert tdeldata.exceptAll(tdeldata_expected).count() == 0
#

Writing test_programaestudiante.py


In [11]:
#@title spark-submit programaestudiante.py { vertical-output: true, form-width: "50%", display-mode: "both" }
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 450})'''))
!spark-submit programaestudiante.py persona*.json
!python -m pytest -vv


<IPython.core.display.Javascript object>

21/08/11 17:51:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/08/11 17:51:32 INFO SparkContext: Running Spark version 3.1.2
21/08/11 17:51:32 INFO ResourceUtils: ==============================================================
21/08/11 17:51:32 INFO ResourceUtils: No custom resources configured for spark.driver.
21/08/11 17:51:32 INFO ResourceUtils: ==============================================================
21/08/11 17:51:32 INFO SparkContext: Submitted application: App#1
21/08/11 17:51:32 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
21/08/11 17:51:32 INFO ResourceProfile

In [9]:
# %%capture
# from procesamientodatos import *

# dfinicial = cargar_datos(['persona*.json'])
# # dfinicial[0] = dfinicial[0].filter('precio_kilometro > 450')
# # dfinicial[0] = dfinicial[0].withColumn('codigo_postal_destino', F.when(F.col('kilometros')<10, '').otherwise(F.col('codigo_postal_destino')))
# # dfinicial[0] = dfinicial[0].withColumn('codigo_postal_origen', F.when(F.col('kilometros')>20, '').otherwise(F.col('codigo_postal_origen')))
# proceso = generar_tablas(dfinicial)
# # archivos = almacenar_tablas(proceso, ['total_viajes.csv', 'total_ingresos.csv', 'metricas.csv'])

In [10]:
# type(proceso)
# len(proceso)
# dfinicial[0].show(50)
# list(map(lambda x: x.show(50), proceso))
# [[i[:] for i in proceso[j].collect()] for j in range(len(proceso))]

---